# Data cleaning 

In [548]:
import pandas as pd

In [549]:
from use_cases.contributions import create_table_contributions

from use_cases.emotions import create_table_emotions
from use_cases.personal_needs import create_table_personal_needs
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data Surveys

In [550]:
chunksize   = 100 # Just for debugging
survey_path = './data/BBDD_Dialogos_2021_01_27.csv' 
survey = pd.read_csv(survey_path, chunksize=chunksize, low_memory=False)
for survey in survey: break
    
ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

In [551]:
survey.sample(2)
ind_online_survey.sample(2)

,Submission Date,RUN,Edad,Comuna,1. ¿Cuál es el nivel de educación alcanzado por Usted?,2. ¿Cuál es el nivel de educación que alcanzó el jefe de su hogar? (sólo si se trata de alguien distinto a Usted),1 >> Emociones / Sentimientos / Sensaciones,1 >> Explique lo mencionado,2 >> Emociones / Sentimientos / Sensaciones,2 >> Explique lo mencionado,...,2 >> Rol del actor social (Describa),3 >> Necesidades del país identificadas,3 >> Rol del Estado (Describa),"3 >> Actor social (empresa, organizaciones sociales, medios de comunicación, comunidad, etc)",3 >> Rol del actor social (Describa),5. Pregunta: ¿Cuál es mi contribución personal para construir el Chile que queremos?,Comuna.1,"Comentarios finales: Si tiene alguna observación o sugerencia, en este espacio puede indicarla.",Región,Región.1
1584,2019/11/29 15:52:33,ebf80e0c64045cdcc8922626638e45af,44,La Serena,Universitaria completa.,Yo soy el jefe de hogar.,Miedo,Miedo del país que le dejare a mi hijo,Pena,Pena por las personas que han perdido sus fuen...,...,pago obligatorio de seguro de salud a los trab...,Mejorar sueldo minimo,Legislar para aumentarlo,empresa debe tener ética social y de justicia,El sueldo minimo solo debe ser referencial. En...,Estoy segura que el día que entendamos lo que ...,NaN,No participare en cabildos porque siento que e...,NaN,Coquimbo
470,2020/01/15 11:09:37,58b5eafe5e4edfd031b4a476cee99ce0,47,NaN,"Post Grado (Master, Doctor o equivalente)",Yo soy el jefe de hogar.,desesperanzado,Al ver tanta violencia y division entre los ch...,triste,"al ver tanta ignarancia y falta de educacion, ...",...,NaN,NaN,NaN,NaN,NaN,Estoy consciente de mis deberes y trato de cum...,Concepción,NaN,Región del Biobío,NaN


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [552]:
persons_table = create_table_persons(survey)

In [553]:
persons_table.sample(3)

,id,diag_id,age,sex,level,comuna_id,age_range
99,299,enc_u_4595903120153792064,38,f,universitaria_completa,7101,30-45
82,481,enc_u_4596766050112223446,33,h,media_completa,1,30-45
37,436,enc_u_4598383170151924860,77,f,basica_incompleta,7101,>=60


### Dialogues

In [554]:
dialogues_table = create_table_dialogues(survey)

In [555]:
dialogues_table.sample(3)

,id,date,init_time,end_time,location,address,comuna_id,n_members,group_name
85,enc_u_4596619275164164128,03-02-2020,15:00:00,17:00:00,consultorio cajual la serena,colon 225,4101,9,usuarios de oficina movil de bienes nacionales
2,enc_u_4602777200118509403,14-02-2020,10:20:00,12:20:00,municipalidad de cunco,pedro aguirre cerda 600,9103,5,usuarios de oficina movil de bienes nacionales
83,enc_u_4596763700997702847,09-03-2020,18:30:00,20:30:00,barros luco y vecinos,barros luco #2247,15101,5,usuarios de oficina movil de bienes nacionales


### Individuals

In [11]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey)

CPU times: user 7.37 s, sys: 28.8 ms, total: 7.4 s
Wall time: 7.35 s


In [12]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,online,age_range
2059,3094162,15-01-2020,65,4301,Educación básica incompleta o inferior,,>=60
6777,3449396,27-01-2020,59,6301,Media incompleta (incluyendo Media Técnica),,45-60


### Emotions

<img src="diagrams/q1.png" width=730 height=520 />

In [560]:
%%time
emotions_table = create_table_emotions(survey,ind_survey_path, ind_online_survey)

CPU times: user 39.4 s, sys: 2.29 s, total: 41.7 s
Wall time: 41.7 s


In [562]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online,source_id
3724,3849,,,regular,[regular],,altos y bajos economia del pais,"[altos, bajos, economia, pais]",0,3172624
24,24,ENC_U_4599526090195321552,,temor,[temor],temor,a la situacion que podria continuar o terminar...,"[situacion, podria, continuar, terminar, peor]",0,


### Country Needs

In [565]:
%%time
country_needs = create_table_country_needs(survey, ind_survey, ind_survey_path, ind_online_survey_path)

CPU times: user 1min 13s, sys: 3.03 s, total: 1min 16s
Wall time: 1min 16s


In [572]:
country_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
2595,3297,,3395725,dialogo,[dialogo],,,,"capacidad de escucha de todos los sectores, ac...","[capacidad, escucha, sectores, aceptacion, dif...",estado,,0
14086,30662,,4280222,salud,[salud],,,,,,,3,0
11861,41683,,3631392,pension digna,"[pension, digna]",,,,,,,,0


### Family/PersonalNeeds

In [575]:
%%time
personal_needs = create_table_personal_needs(survey, ind_survey, ind_online_survey)

CPU times: user 58.9 s, sys: 4 s, total: 1min 2s
Wall time: 1min 2s


In [576]:
personal_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
10954,22934,,5336323,sueldos bajos,"[sueldos, bajos]",no cubre las necesidades de las familias y un ...,"[cubre, necesidades, familias, standar, calida...",sueldos bajos,2.0,0
1365,26198,,62117f5b2140e724c9728345caf6feb1,costos de salud por enfermedades,"[costos, salud, enfermedades]","en seis meses mas, si quiero mantener mi plan ...","[seis, meses, mas, si, quiero, mantener, plan,...",costos de salud por enfermedades,2.0,1
710,25544,,f62d0382b6ce69991a27c1a40b523f20,pensiones bajas.,"[pensiones, bajas]",mi padre trabajo como profesional toda su vida...,"[padre, trabajo, profesional, toda, vida, sect...",pensiones bajas.,2.0,1


### Contributions

In [577]:
contributions = create_table_contributions(survey, ind_survey, ind_online_survey)

In [578]:
contributions.sample(3)

,id,diag_id,ind_id,text,tokens,macro,is_online
5377,5877,,3226252,colaborar y unir a las personas para que todos...,"[colaborar, unir, personas, aportemos, granito...",colaborar y unir a las personas para que todos...,0
6999,7499,,3398834,trabajo como voluntario en la asociacion de sc...,"[trabajo, voluntario, asociacion, scout, chile]",trabajo como voluntario en la asociacion de sc...,0
10979,11479,,5337903,esperar las soluciones,"[esperar, soluciones]",esperar las soluciones,0
